<a href="https://colab.research.google.com/github/mczarkowska/BIOS_lab2/blob/main/laboratorium2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install biopython

from google.colab import files

uploaded = files.upload()

from Bio.PDB import PDBParser
import numpy as np
from Bio.PDB.vectors import calc_dihedral

# Słownik atomów dla kątów torsyjnych w RNA
# Zaktualizowany słownik atomów dla kątów torsyjnych w RNA
torsion_atoms = {
    'alpha': ('O3\'', 'P', 'O5\'', 'C5\''),
    'beta': ('P', 'O5\'', 'C5\'', 'C4\''),
    'gamma': ('O5\'', 'C5\'', 'C4\'', 'C3\''),
    'delta': ('C5\'', 'C4\'', 'C3\'', 'O3\''),
    'epsilon': ('C4\'', 'C3\'', 'O3\'', 'P'),
    'zeta': ('C3\'', 'O3\'', 'P', 'O5\''),
    'chi_pyrimidines': ('O4\'', 'C1\'', 'N1', 'C2'), # Dla pirymidyn
    'chi_purines': ('O4\'', 'C1\'', 'N9', 'C2') # Dla puryn
}

# Funkcja do obliczania kąta torsyjnego między czterema atomami
def get_torsion_angle(atom1, atom2, atom3, atom4):
    return calc_dihedral(atom1.get_vector(), atom2.get_vector(), atom3.get_vector(), atom4.get_vector())

# Funkcja do wyboru odpowiedniego zestawu atomów dla kąta chi w zależności od bazy
def get_chi_atoms(residue, torsion_atoms):
    if 'N9' in [atom.get_name() for atom in residue.get_unpacked_list()]:  # Puryny mają atom N9
        return [residue[atom_name] for atom_name in torsion_atoms['chi_purines']]
    else:  # Dla pirymidyn
        return [residue[atom_name] for atom_name in torsion_atoms['chi_pyrimidines']]

# Funkcja do wyznaczania kątów torsyjnych dla RNA
def calculate_rna_torsion_angles(structure_id, pdb_path):
    parser = PDBParser(QUIET=True)
    structure = parser.get_structure(structure_id, pdb_path)
    angle_matrix = []

    for model in structure:
        for chain in model:
            for residue in chain:
                angles = []
                for angle_name, atoms in torsion_atoms.items():
                    try:
                        # Dla 'chi' różne atomy dla puryn i pirymidyn
                        if angle_name == 'chi':
                            atom_names = list(atoms)
                            if residue.resname.strip() in ['A', 'G']:
                                atom_names[2] = 'N9'
                            atoms = tuple(atom_names)

                        # Pobierz atomy dla kąta
                        atoms_objs = [residue[atom_name] for atom_name in atoms if atom_name in residue]
                        # Oblicz kąt torsyjny tylko jeśli wszystkie atomy są obecne
                        if len(atoms_objs) == 4:
                            angle = get_torsion_angle(*atoms_objs)
                            angles.append(np.degrees(angle))
                        else:
                            angles.append(None)
                    except Exception as e:
                        angles.append(None)
                        print(f"Error calculating {angle_name}: {e}")
                angle_matrix.append(angles)

    # Zapisz wyniki do pliku
    output_file = "/content/rna_torsion_angles.csv"
    np.savetxt(output_file, np.array(angle_matrix), delimiter=",", fmt='%s')
    return output_file

# Wyznaczenie kątów dla RNA o PDB ID 1EHZ
angles_rna = calculate_rna_torsion_angles("1EHZ", "1EHZ.pdb")
print("Kąty torsyjne zostały obliczone i zapisane do pliku:", angles_rna)

In [ ]:
from google.colab import files
files.download('/content/rna_torsion_angles.csv')

In [ ]:
!pip install biopython

from google.colab import files


import matplotlib.pyplot as plt
import numpy as np
from google.colab import files
from Bio.PDB import PDBParser, PPBuilder

# Upload the PDB file
uploaded = files.upload()
pdb_filename = next(iter(uploaded))

def plot_ramachandran(structure_id, pdb_filename):
    parser = PDBParser(QUIET=True)
    structure = parser.get_structure(structure_id, pdb_filename)

    phi_psi_list = []
    colors = []

    # Use PPBuilder to get polypeptides from the structure
    ppb = PPBuilder()
    for pp in ppb.build_peptides(structure):
        for phi, psi in pp.get_phi_psi_list():
            if phi and psi:  # Check if phi and psi angles are defined (not None)
                phi_deg = np.degrees(phi)
                psi_deg = np.degrees(psi)
                phi_psi_list.append((phi_deg, psi_deg))

                # Simple secondary structure assignment by angle regions
                if -100 <= phi_deg <= -40 and -70 <= psi_deg <= -30:
                    colors.append('red')  # Alpha helix
                elif -180 <= phi_deg <= -120 and 90 <= psi_deg <= 180:
                    colors.append('blue')  # Beta sheet
                else:
                    colors.append('green')  # Other

    # Convert list of tuples to a numpy array
    phi_psi_array = np.array(phi_psi_list)
    phi, psi = phi_psi_array.T  # Transpose to get phi and psi in separate arrays

    # Create Ramachandran plot
    plt.figure(figsize=(8, 6))
    plt.scatter(phi, psi, c=colors, s=10)
    plt.title(f'Ramachandran Plot for {structure_id}')
    plt.xlabel('Phi (°)')
    plt.ylabel('Psi (°)')
    plt.xlim(-180, 180)
    plt.ylim(-180, 180)
    plt.show()

# Call the function with the uploaded PDB file
plot_ramachandran("1MBO", pdb_filename)
